In [185]:
import pandas as pd
import os


In [186]:
LLM_score_thresh = 0.8
enrichr_adj_pval_thresh = 0.05
enrichr_JI_thresh = 0.1
coverage_thresh = 0.2

In [187]:

import os
import pandas as pd

# Read the TSV file into a Pandas DataFrame
LLM_genes_DF = pd.read_csv("data/omics_revamped_LLM_Enrichr_simVals_DF.tsv", delimiter="\t")
#LLM_genes_DF.shape
LLM_genes_DF.head(5)

,Unnamed: 0.1,Unnamed: 0,Source,GeneSetID,GeneSetName,GeneList,n_Genes,LLM Name,LLM Analysis,Score,Rank,Overlap,P-value,Adjusted P-value,Genes,Term,GO term,GO ID,GO_term_genes,LLM_name_GO_term_sim
0,0,0,NeST,Cluster1-10,Cluster1-10,CTRL HSD17B14 KIAA0232 PAQR8 PLA2G1B RNF145 SG...,12,Lipid Metabolism and Membrane Dynamics,"1. CTRL, or carboxyl-terminal esterase/lipase,...",0.85,0,2/21,0.000069,0.008264,PLA2G1B;HSD17B14,Lipid Catabolic Process (GO:0016042),Lipid Catabolic Process,GO:0016042,CES1 PNLIPRP1 PLA2G1B CES1P1 AOAH HSD17B14 APO...,0.603573
1,1,0,NeST,Cluster1-11,Cluster1-11,LMF1 MFHAS1 MR1 PLA2G1B RASL11A RNF145 SLC2A6 ...,12,Lipid Metabolism and Membrane Trafficking,1. LMF1 (Lipase Maturation Factor 1) is crucia...,0.85,0,2/85,0.001146,0.050873,PLA2G1B;MR1,Defense Response To Gram-positive Bacterium (G...,Defense Response To Gram-positive Bacterium,GO:0050830,GBP7 CHGA GBP6 GSDMD DEFB104A H2BC8 RNASE12 RN...,0.317063
2,2,0,NeST,Cluster1-12,Cluster1-12,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,12,Lipid Metabolism and Receptor-Mediated Endocyt...,"1. AMY2B (Amylase, alpha 2B) is an enzyme that...",0.88,0,4/189,0.000004,0.000671,VLDLR;SORL1;LRP8;LDLR,Endocytosis (GO:0006897),Endocytosis,GO:0006897,RALBP1 SCARA5 TBC1D2B AP1S1 TSC2 ARHGAP27 TICA...,0.607433
3,3,1,NeST,Cluster1-12,Cluster1-12,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,12,Lipid Metabolism and Receptor-Mediated Endocyt...,"1. AMY2B (Amylase, alpha 2B) is an enzyme that...",0.88,1,2/6,0.000005,0.000671,LRPAP1;LDLR,Negative Regulation Of Lipoprotein Particle Cl...,Negative Regulation Of Lipoprotein Particle Cl...,GO:0010985,KHSRP APOC3 APOC2 PCSK9 LDLR LRPAP1,0.440780
4,4,2,NeST,Cluster1-12,Cluster1-12,AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP P...,12,Lipid Metabolism and Receptor-Mediated Endocyt...,"1. AMY2B (Amylase, alpha 2B) is an enzyme that...",0.88,2,4/233,0.000008,0.000671,RELN;VLDLR;SORL1;LRP8,Positive Regulation Of Developmental Process (...,Positive Regulation Of Developmental Process,GO:0051094,ACVR1 KRT2 ISL1 SMARCA2 SMARCA4 CDC20 NEURL1 T...,0.150215


In [188]:
# In 20 cases, there is no overlap with any GO_term's gene set
# Use this function to find those cases
def find_non_string(df):
    nonstring = 0
    for index, row in df.iterrows():
        go_term_genes = row["GO_term_genes"]
        if not isinstance(go_term_genes, str):
            nonstring += 1
        print(f'index: {index}\n{row}\n---------\n')
    print(nonstring)

In [200]:
# functions to compute the Jaccard Index and coverage for the query gene list and the GO term genes
def get_JI(GeneList: str, enrichr_genes) -> float:
    # Check if enrichr_genes is a string
    if not isinstance(enrichr_genes, str):
        #print(f"Warning: enrichr_genes is not a string: {enrichr_genes}")
        return 0
    
    # Assume separation is the same
    geneSetGenes = GeneList.split(" ")
    enrichRGenes = enrichr_genes.split(" ")
    
    # Calculate the Jaccard Index (JI)
    intersection = set(geneSetGenes).intersection(enrichRGenes)
    union = set(geneSetGenes).union(enrichRGenes)
    
    JI = len(intersection) / len(union) if len(union) != 0 else 0
    
    return JI

def get_coverage(GeneList: str, enrichr_genes) -> float:
    # Check if enrichr_genes is a string
    if not isinstance(enrichr_genes, str):
        #print(f"Warning: enrichr_genes is not a string: {enrichr_genes}")
        return 0
    
    # Assume separation is the same
    geneSetGenes = GeneList.split(" ")
    enrichRGenes = enrichr_genes.split(" ")
    
    # Calculate the intersection
    intersection = set(geneSetGenes).intersection(enrichRGenes)
    
    # Calculate the coverage
    coverage = len(intersection) / len(geneSetGenes) if len(geneSetGenes) != 0 else 0
    
    return coverage

def select_rows_and_columns(df, number_of_rows=None):
    # Specify the ordered list of column names you want to return
    ordered_column_names = [
        'n_Genes', 'GeneList', 'LLM Name', 'Score', 'GO term', 'GO_term_genes',
        'Genes', 'Overlap', 'Adjusted P-value', 'coverage', 'enrichr_JI', 
        'LLM_success_TF', 'enrichr_success_TF'
    ]
    
    # If number_of_rows is not specified, select all rows
    if number_of_rows is None:
        result_df = df.loc[:, ordered_column_names]
    else:
        result_df = df.loc[:, ordered_column_names].head(number_of_rows)
    
    return result_df

In [190]:
# Apply the get_JI and get_coverage functions to each row and add new columns to the DataFrame
LLM_genes_DF['enrichr_JI'] = LLM_genes_DF.apply(lambda row: get_JI(row['GeneList'], 
                                                                   row['GO_term_genes']), 
                                                                   axis=1)

LLM_genes_DF['coverage'] = LLM_genes_DF.apply(lambda row: get_coverage(row['GeneList'], 
                                                                   row['GO_term_genes']), 
                                                                   axis=1)
LLM_genes_DF.shape

(11310, 22)

~~~

get_JI = function(GeneList, enrichr_genes){
    # assume separation is the same
    geneSetGenes = str_split(string = GeneList, pattern = " ")[[1]]
    enrichRGenes = str_split(string = enrichr_genes, pattern = " ")[[1]]
    JI = length(intersect(geneSetGenes, enrichRGenes))/ length(union(geneSetGenes, enrichRGenes))
    
    return(JI)
    }   

-----------------------------

LLM_genes_DF = LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes))

LLM_genes_reduced_DF = LLM_genes_DF %>%
group_by("Source", "GeneSetID", "GeneSetName")

reduced_LLM_genes_DF = LLM_genes_DF %>%
group_by(Source, GeneSetID, GeneSetName, GeneList) %>%
slice(which.max(enrichr_JI))

reduced_LLM_genes_DF = reduced_LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes)) %>%
mutate(LLM_success_TF = ifelse(Score < 0.8, FALSE, TRUE),
       enrichr_success_TF = ifelse(((`Adjusted P-value` > enrichr_adj_pval_thresh) | (enrichr_JI < enrichr_JI_thresh)), FALSE, TRUE))

-------------------------------
Single cell - is this the same?

reduced_LLM_genes_DF = LLM_genes_DF %>%
group_by(Source, GeneSetID, GeneSetName, GeneList) %>%
slice(which.max(enrichr_JI))

reduced_LLM_genes_DF = reduced_LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes)) %>%
mutate(LLM_success_TF = ifelse(Score < 0.8, FALSE, TRUE),
       enrichr_success_TF = ifelse(((`Adjusted P-value` > enrichr_adj_pval_thresh) | (enrichr_JI < enrichr_JI_thresh)), FALSE, TRUE))

------------------------------
figure from the paper:

LLM_success_TF      True   False  Total
enrichr_success_TF              
True                  36      4     40
False                 97    163    260 
Total                133    167    300

The LLM succeeds 133 out of 300 time
Enrichment + JI succeeds 40 out of 300
~~~


In [191]:
# Reproducing the original analysis:

# create a DataFrame reduced_LLM_genes_by_JI_DF containing the rows with the highest enrichr_JI 
# for each combination of Source, GeneSetID, GeneSetName, GeneList.
grouped = LLM_genes_DF.groupby(["Source", "GeneSetID", "GeneSetName", "GeneList"])

# Define a function to return the row with the maximum 'enrichr_JI' for each group
def get_max_enrichr_JI(df):
    return df.loc[df['enrichr_JI'].idxmax()]

# Apply the function to each group and reset the index
# We take the GO term with the highest Jaccard Index
reduced_LLM_genes_by_JI_DF = grouped.apply(get_max_enrichr_JI).reset_index(drop=True)

# Add the LLM_success_TF and enrichr_success_TF columns
# LLM_success_TF is True if the 'Score' is greater than or equal to LLM_score_thresh
# enrichr_success_TF is False if the 'Adjusted P-value' is less or equal to  enrichr_adj_pval_thresh and the 'enrichr_JI' is greater than or equal to enrichr_JI_thresh
reduced_LLM_genes_by_JI_DF['LLM_success_TF'] = reduced_LLM_genes_by_JI_DF['Score'] >= LLM_score_thresh
reduced_LLM_genes_by_JI_DF['enrichr_success_TF'] = ~((reduced_LLM_genes_by_JI_DF['Adjusted P-value'] > enrichr_adj_pval_thresh) | (reduced_LLM_genes_by_JI_DF['enrichr_JI'] < enrichr_JI_thresh))

# compute the stats for the enrichr results
contingency_table_LLM_vs_enrichr_by_JI = pd.crosstab(reduced_LLM_genes_by_JI_DF['enrichr_success_TF'], reduced_LLM_genes_by_JI_DF['LLM_success_TF'], rownames=['enrichr_success_TF'], colnames=['LLM_success_TF'])
# Reorder the rows and columns
contingency_table_LLM_vs_enrichr_by_JI = contingency_table_LLM_vs_enrichr_by_JI.reindex(index=[True, False], columns=[True, False])
## Add the total column and row
contingency_table_LLM_vs_enrichr_by_JI['Total'] = contingency_table_LLM_vs_enrichr_by_JI.sum(axis=1)
contingency_table_LLM_vs_enrichr_by_JI.loc['Total'] = contingency_table_LLM_vs_enrichr_by_JI.sum(axis=0)

# Print the contingency table
print(contingency_table_LLM_vs_enrichr_by_JI)

LLM_success_TF      True  False  Total
enrichr_success_TF                    
True                  36      4     40
False                 97    163    260
Total                133    167    300


In [192]:
# ChatGPT correction to the original analysis translation
# adds a step done in the R code but which seems unnecessary

# Grouping the DataFrame by the specified columns
grouped = LLM_genes_DF.groupby(["Source", "GeneSetID", "GeneSetName", "GeneList"])

# Define a function to return the row with the maximum 'enrichr_JI' for each group
def get_max_enrichr_JI(df):
    return df.loc[df['enrichr_JI'].idxmax()]

# Apply the function to each group and reset the index
reduced_LLM_genes_by_JI_DF = grouped.apply(get_max_enrichr_JI).reset_index(drop=True)

# Recalculate enrichr_JI for each row
reduced_LLM_genes_by_JI_DF['enrichr_JI'] = reduced_LLM_genes_by_JI_DF.apply(lambda row: get_JI(row['GeneList'], row['GO_term_genes']), axis=1)

# Add the 'LLM_success_TF' and 'enrichr_success_TF' columns
reduced_LLM_genes_by_JI_DF['LLM_success_TF'] = reduced_LLM_genes_by_JI_DF['Score'] >= LLM_score_thresh
reduced_LLM_genes_by_JI_DF['enrichr_success_TF'] = ~((reduced_LLM_genes_by_JI_DF['Adjusted P-value'] > enrichr_adj_pval_thresh) | (reduced_LLM_genes_by_JI_DF['enrichr_JI'] < enrichr_JI_thresh))

# compute the stats for the enrichr results
contingency_table_LLM_vs_enrichr_by_JI = pd.crosstab(reduced_LLM_genes_by_JI_DF['enrichr_success_TF'], reduced_LLM_genes_by_JI_DF['LLM_success_TF'], rownames=['enrichr_success_TF'], colnames=['LLM_success_TF'])
# Reorder the rows and columns
contingency_table_LLM_vs_enrichr_by_JI = contingency_table_LLM_vs_enrichr_by_JI.reindex(index=[True, False], columns=[True, False])
## Add the total column and row
contingency_table_LLM_vs_enrichr_by_JI['Total'] = contingency_table_LLM_vs_enrichr_by_JI.sum(axis=1)
contingency_table_LLM_vs_enrichr_by_JI.loc['Total'] = contingency_table_LLM_vs_enrichr_by_JI.sum(axis=0)

# Print the contingency table
print(contingency_table_LLM_vs_enrichr_by_JI)

LLM_success_TF      True  False  Total
enrichr_success_TF                    
True                  36      4     40
False                 97    163    260
Total                133    167    300


In [193]:
# Alternative analysis in which the reduced DF is based on the minimum of Adjusted P-value

# create a DataFrame reduced_LLM_genes_by_apv_DF containing the rows with the lowest 'Adjusted P-value' 
# for each combination of Source, GeneSetID, GeneSetName, GeneList.

# Group the DataFrame by the specified columns
grouped = LLM_genes_DF.groupby(["Source", "GeneSetID", "GeneSetName", "GeneList"])

# Define a function to return the row with the minimum 'Adjusted P-value' for each group
def get_min_adj_p_value(df):
    return df.loc[df['Adjusted P-value'].idxmin()]

# Apply the function to each group and reset the index
reduced_LLM_genes_by_apv_DF = grouped.apply(get_min_adj_p_value).reset_index(drop=True)

# Add the LLM_success_TF and enrichr_success_TF columns
reduced_LLM_genes_by_apv_DF['LLM_success_TF'] = reduced_LLM_genes_by_apv_DF['Score'] >= LLM_score_thresh

# The resulting DataFrame will have an enrichr_success_TF column where the values are True if both conditions 
# (Adjusted P-value >= enrichr_adj_pval_thresh) and (enrichr_JI <= enrichr_JI_thresh) are met. 
# Otherwise, it will be False.
reduced_LLM_genes_by_apv_DF['enrichr_success_TF'] = ~((reduced_LLM_genes_by_apv_DF['Adjusted P-value'] > enrichr_adj_pval_thresh) | (reduced_LLM_genes_by_apv_DF['enrichr_JI'] < enrichr_JI_thresh))

# make the LLM vs enrichr contingency table apv
contingency_table_LLM_vs_enrichr_by_apv = pd.crosstab(reduced_LLM_genes_by_apv_DF['enrichr_success_TF'], 
                                                      reduced_LLM_genes_by_apv_DF['LLM_success_TF'], 
                                                      rownames=['enrichr_success_TF'], 
                                                      colnames=['LLM_success_TF'])

# Reorder the rows and columns
contingency_table_LLM_vs_enrichr_by_apv = contingency_table_LLM_vs_enrichr_by_apv.reindex(index=[True, False], columns=[True, False])

## Add the total column and row
contingency_table_LLM_vs_enrichr_by_apv['Total'] = contingency_table_LLM_vs_enrichr_by_apv.sum(axis=1)
contingency_table_LLM_vs_enrichr_by_apv.loc['Total'] = contingency_table_LLM_vs_enrichr_by_apv.sum(axis=0)

# Print the contingency table
print(contingency_table_LLM_vs_enrichr_by_apv)



LLM_success_TF      True  False  Total
enrichr_success_TF                    
True                  21      4     25
False                112    163    275
Total                133    167    300


In [194]:
# remove the JI requirement

# Alternative analysis in which the reduced DF is based on the minimum of Adjusted P-value

# create a DataFrame reduced_LLM_genes_by_apv_DF containing the rows with the lowest 'Adjusted P-value' 
# for each combination of Source, GeneSetID, GeneSetName, GeneList.

# Group the DataFrame by the specified columns
grouped = LLM_genes_DF.groupby(["Source", "GeneSetID", "GeneSetName", "GeneList"])

# Define a function to return the row with the minimum 'Adjusted P-value' for each group
def get_min_adj_p_value(df):
    return df.loc[df['Adjusted P-value'].idxmin()]

# Apply the function to each group and reset the index
reduced_LLM_genes_by_apv_DF = grouped.apply(get_min_adj_p_value).reset_index(drop=True)

# Add the LLM_success_TF and enrichr_success_TF columns
reduced_LLM_genes_by_apv_DF['LLM_success_TF'] = reduced_LLM_genes_by_apv_DF['Score'] >= LLM_score_thresh

# The resulting DataFrame will have an enrichr_success_TF column where the values are True if both conditions 
# (Adjusted P-value >= enrichr_adj_pval_thresh) and (enrichr_JI <= enrichr_JI_thresh) are met. 
# Otherwise, it will be False.
reduced_LLM_genes_by_apv_DF['enrichr_success_TF'] = ~(reduced_LLM_genes_by_apv_DF['Adjusted P-value'] > enrichr_adj_pval_thresh)

#reduced_LLM_genes_by_apv_DF['enrichr_success_TF'] = (reduced_LLM_genes_by_apv_DF['Adjusted P-value'] <= enrichr_adj_pval_thresh) & (reduced_LLM_genes_by_apv_DF['enrichr_JI'] >= enrichr_JI_thresh)

# The first three rows resulting DataFrame
# reduced_LLM_genes_by_apv_DF[0:7]
# contingency_table_LLM_by_apv = pd.crosstab(reduced_LLM_genes_by_apv_DF['LLM_success_TF'], reduced_LLM_genes_by_apv_DF['Source'])

# Display the contingency table
# print(contingency_table_LLM_by_apv)

# print("\n")

# contingency_table_enrichr_by_apv = pd.crosstab(reduced_LLM_genes_by_apv_DF['enrichr_success_TF'], reduced_LLM_genes_by_apv_DF['Source'])
# Display the contingency table
# print(contingency_table_enrichr_by_apv)

# print("\n")

# make the LLM vs enrichr contingency table apv
contingency_table_LLM_vs_enrichr_by_apv = pd.crosstab(reduced_LLM_genes_by_apv_DF['enrichr_success_TF'], 
                                                      reduced_LLM_genes_by_apv_DF['LLM_success_TF'], 
                                                      rownames=['enrichr_success_TF'], 
                                                      colnames=['LLM_success_TF'])

# Reorder the rows and columns
contingency_table_LLM_vs_enrichr_by_apv = contingency_table_LLM_vs_enrichr_by_apv.reindex(index=[True, False], columns=[True, False])

## Add the total column and row
contingency_table_LLM_vs_enrichr_by_apv['Total'] = contingency_table_LLM_vs_enrichr_by_apv.sum(axis=1)
contingency_table_LLM_vs_enrichr_by_apv.loc['Total'] = contingency_table_LLM_vs_enrichr_by_apv.sum(axis=0)

# Print the contingency table
print(contingency_table_LLM_vs_enrichr_by_apv)



LLM_success_TF      True  False  Total
enrichr_success_TF                    
True                 117    103    220
False                 16     64     80
Total                133    167    300


In [195]:
# Group the DataFrame by the specified columns
grouped = LLM_genes_DF.groupby(["Source", "GeneSetID", "GeneSetName", "GeneList"])

# Define a function to filter and select the appropriate row
def filter_and_select(df):
    # Filter rows that pass both Enrichr requirements
    filtered = df[(df['enrichr_JI'] >= enrichr_JI_thresh) & (df['Adjusted P-value'] <= enrichr_adj_pval_thresh)]
    
    if not filtered.empty:
        # If there are rows that pass both requirements, select the one with the lowest adjusted p-value
        return filtered.loc[filtered['Adjusted P-value'].idxmin()]
    else:
        # If no rows pass both requirements, select the one with the lowest adjusted p-value in the original group
        return df.loc[df['Adjusted P-value'].idxmin()]

# Apply the function to each group and reset the index
reduced_LLM_genes_by_apv_best_DF = grouped.apply(filter_and_select).reset_index(drop=True)

# Recalculate enrichr_JI for each row
reduced_LLM_genes_by_apv_best_DF['enrichr_JI'] = reduced_LLM_genes_by_apv_best_DF.apply(lambda row: get_JI(row['GeneList'], row['GO_term_genes']), axis=1)

# Add the 'LLM_success_TF' and 'enrichr_success_TF' columns
reduced_LLM_genes_by_apv_best_DF['LLM_success_TF'] = reduced_LLM_genes_by_apv_best_DF['Score'] >= LLM_score_thresh
reduced_LLM_genes_by_apv_best_DF['enrichr_success_TF'] = ~((reduced_LLM_genes_by_apv_best_DF['Adjusted P-value'] > enrichr_adj_pval_thresh) | (reduced_LLM_genes_by_apv_best_DF['enrichr_JI'] < enrichr_JI_thresh))

# make the LLM vs enrichr contingency table apv
contingency_table_LLM_vs_enrichr_by_apv_best = pd.crosstab(reduced_LLM_genes_by_apv_best_DF['enrichr_success_TF'], 
                                                      reduced_LLM_genes_by_apv_best_DF['LLM_success_TF'], 
                                                      rownames=['enrichr_success_TF'], 
                                                      colnames=['LLM_success_TF'])

# Reorder the rows and columns
contingency_table_LLM_vs_enrichr_by_apv_best = contingency_table_LLM_vs_enrichr_by_apv_best.reindex(index=[True, False], columns=[True, False])

## Add the total column and row
contingency_table_LLM_vs_enrichr_by_apv_best['Total'] = contingency_table_LLM_vs_enrichr_by_apv_best.sum(axis=1)
contingency_table_LLM_vs_enrichr_by_apv_best.loc['Total'] = contingency_table_LLM_vs_enrichr_by_apv_best.sum(axis=0)

# Print the contingency table
print(contingency_table_LLM_vs_enrichr_by_apv_best)

LLM_success_TF      True  False  Total
enrichr_success_TF                    
True                  36      4     40
False                 97    163    260
Total                133    167    300


In [201]:
coverage_thresh = 0.15

# Define a function to filter and select the appropriate row
def filter_and_select_coverage(df):
    # Filter rows that pass both Enrichr requirements
    filtered = df[(df['coverage'] >= coverage_thresh) & (df['Adjusted P-value'] <= enrichr_adj_pval_thresh)]
    
    if not filtered.empty:
        # If there are rows that pass both requirements, select the one with the lowest adjusted p-value
        return filtered.loc[filtered['Adjusted P-value'].idxmin()]
    else:
        # If no rows pass both requirements, select the one with the lowest adjusted p-value in the original group
        return df.loc[df['Adjusted P-value'].idxmin()]

# Grouping the DataFrame by the specified columns
grouped = LLM_genes_DF.groupby(["Source", "GeneSetID", "GeneSetName", "GeneList"])

# Apply the function to each group and reset the index
reduced_LLM_genes_by_coverage_DF = grouped.apply(filter_and_select_coverage).reset_index(drop=True)

# Recalculate coverage for each row
reduced_LLM_genes_by_coverage_DF['coverage'] = reduced_LLM_genes_by_coverage_DF.apply(lambda row: get_coverage(row['GeneList'], row['GO_term_genes']), axis=1)

# Add the 'LLM_success_TF' and 'enrichr_success_TF' columns
reduced_LLM_genes_by_coverage_DF['LLM_success_TF'] = reduced_LLM_genes_by_coverage_DF['Score'] >= LLM_score_thresh
reduced_LLM_genes_by_coverage_DF['enrichr_success_TF'] = ~((reduced_LLM_genes_by_coverage_DF['Adjusted P-value'] > enrichr_adj_pval_thresh) | (reduced_LLM_genes_by_coverage_DF['coverage'] < coverage_thresh))

# Make the LLM vs enrichr contingency table using coverage
contingency_table_LLM_vs_enrichr_by_coverage = pd.crosstab(reduced_LLM_genes_by_coverage_DF['enrichr_success_TF'], 
                                                           reduced_LLM_genes_by_coverage_DF['LLM_success_TF'], 
                                                           rownames=['enrichr_success_TF'], 
                                                           colnames=['LLM_success_TF'])

# Reorder the rows and columns
contingency_table_LLM_vs_enrichr_by_coverage = contingency_table_LLM_vs_enrichr_by_coverage.reindex(index=[True, False], columns=[True, False])

# Add the total column and row
contingency_table_LLM_vs_enrichr_by_coverage['Total'] = contingency_table_LLM_vs_enrichr_by_coverage.sum(axis=1)
contingency_table_LLM_vs_enrichr_by_coverage.loc['Total'] = contingency_table_LLM_vs_enrichr_by_coverage.sum(axis=0)

# Print the contingency table
print(contingency_table_LLM_vs_enrichr_by_coverage)

result_df = select_rows_and_columns(reduced_LLM_genes_by_coverage_DF, 5)
result_df

LLM_success_TF      True  False  Total
enrichr_success_TF                    
True                  91     33    124
False                 42    134    176
Total                133    167    300


,n_Genes,GeneList,LLM Name,Score,GO term,GO_term_genes,Genes,Overlap,Adjusted P-value,coverage,enrichr_JI,LLM_success_TF,enrichr_success_TF
0,24,CITED2 COL5A1 CRABP2 KCTD12 MDFIC MMP2 NRP1 OR...,Cellular Matrix Remodeling and Tissue Development,0.85,Regulation Of Peroxisome Proliferator Activate...,ASXL2 ASXL1 PTGIS BMP2 CITED2 GPS2 LMO3 PLIN5 ...,CITED2;TWIST1,2/10,0.023830,0.083333,0.062500,True,False
1,47,1060P11.3 ADM AHR AMIGO2 ARL4C ATP10D CAV2 CD4...,Cellular Adhesion and Extracellular Matrix Int...,0.85,Purine Ribonucleoside Monophosphate Catabolic ...,PNP NT5C1A NT5E HPRT1 SORD PRTFDC1 NT5C2,NT5E;HPRT1,2/7,0.059507,0.042553,0.038462,True,False
2,59,ABAT ASS1 CHI3L1 CHST2 CLDN3 EIF5B FRZB GAL HE...,System of unrelated proteins,0.00,Response To Steroid Hormone,MED1 TGFB1 PAQR7 TGFB3 PAQR8 TGFB2 HSPA8 NR1H3...,CAV1;AKR1B1;THBS1,3/29,0.047162,0.050847,0.035294,False,False
3,9,AMIGO2 AREG GAS6 GPR37 IFT57 PELI1 SQLE AKAP12...,System of unrelated proteins,0.00,Positive Regulation Of Protein Tyrosine Kinase...,PTPN1 IL34 ACE EGF FBXW7 SRCIN1 PRNP AGT EREG ...,GAS6;AREG,2/38,0.023945,0.222222,0.044444,False,True
4,15,1060P11.3 CPE EFNB2 HIST1H2AC IL1R2 INHBB LYPD...,System of unrelated proteins,0.00,SMAD Protein Signal Transduction,GDF11 TGFB1 TGFB3 JUN TGFB2 GDF15 BMP8B BMP8A ...,NUP93;GDF15;INHBB,3/51,0.002024,0.200000,0.047619,False,True


In [197]:
# Write the DataFrames to TSV files
reduced_LLM_genes_by_JI_DF.to_csv("data/omics_revamped_LLM_Enrichr_simVals_failure_by_JI_DF.tsv", sep="\t", index=False)
reduced_LLM_genes_by_apv_DF.to_csv("data/omics_revamped_LLM_Enrichr_simVals_failure_by_apv_DF.tsv", sep="\t", index=False)

In [198]:
# Read the TSV file into a DataFrame
test = pd.read_csv("data/omics_revamped_LLM_Enrichr_simVals_failure_by_apv_DF.tsv", sep="\t")
#test[0:7]
test.shape

(300, 24)

In [199]:
# Translation of the new analysis from Sahar

# She gets this result, which matches my version
# LLM_success_TF      True  False  Total
# enrichr_success_TF                    
# True                  21      4     25
# False                112    163    275
# Total                133    167    300

def process_and_filter(df, enrichr_JI_thresh=0.2, enrichr_adj_pval_thresh=0.05):

    # Group the DataFrame by the specified columns
    grouped = df.groupby(["Source", "GeneSetID", "GeneSetName", "GeneList"])

    # Define a function to process each group
    def filter_and_select(group):
        # Sort the group by 'Adjusted P-value'
        group = group.sort_values(by="Adjusted P-value")

        # Apply the filter conditions
        filter_condition = (
            ((len(group) > 1) & (group["enrichr_JI"] >= enrichr_JI_thresh)) |
            (group["Adjusted P-value"] > enrichr_adj_pval_thresh) |
            (group["Adjusted P-value"] <= enrichr_adj_pval_thresh)
        )

        filtered_group = group[filter_condition]

        # Select the first row of the filtered group
        return filtered_group.head(1)

    # Apply the function to each group and concatenate the results
    reduced_df = grouped.apply(filter_and_select).reset_index(drop=True)
    
    return reduced_df

def process_rows(df):
    # Apply the get_JI function to compute enrichr_JI
    df['enrichr_JI'] = df.apply(lambda row: get_JI(row['GeneList'], row['GO_term_genes']), axis=1)

    # Add the LLM_success_TF and enrichr_success_TF columns
    df['LLM_success_TF'] = df['Score'] >= LLM_score_thresh
    df['enrichr_success_TF'] = ~((df['Adjusted P-value'] > enrichr_adj_pval_thresh) | (df['enrichr_JI'] < enrichr_JI_thresh))

    return df

reduced_LLM_genes_Sahar_new_DF = process_and_filter(LLM_genes_DF, enrichr_JI_thresh=0.1, enrichr_adj_pval_thresh=0.1)


reduced_LLM_genes_Sahar_output_DF = process_rows(reduced_LLM_genes_Sahar_new_DF)


# make the LLM vs enrichr contingency table apv
contingency_table_LLM_vs_enrichr_Sahar = pd.crosstab(reduced_LLM_genes_Sahar_output_DF['enrichr_success_TF'], reduced_LLM_genes_Sahar_output_DF['LLM_success_TF'], rownames=['enrichr_success_TF'], colnames=['LLM_success_TF'])
# Reorder the rows and columns
contingency_table_LLM_vs_enrichr_Sahar = contingency_table_LLM_vs_enrichr_Sahar.reindex(index=[True, False], columns=[True, False])
## Add the total column and row
contingency_table_LLM_vs_enrichr_Sahar['Total'] = contingency_table_LLM_vs_enrichr_Sahar.sum(axis=1)
contingency_table_LLM_vs_enrichr_Sahar.loc['Total'] = contingency_table_LLM_vs_enrichr_Sahar.sum(axis=0)

# Print the contingency table
print(contingency_table_LLM_vs_enrichr_Sahar)

LLM_success_TF      True  False  Total
enrichr_success_TF                    
True                  21      4     25
False                112    163    275
Total                133    167    300


reduced_LLM_genes_DF = read_delim(file = "data/omics_revamped_LLM_Enrichr_simVals_failure_DF.tsv", delim = "\t")

reduced_LLM_genes_DF = reduced_LLM_genes_DF %>%
mutate(enrichr_adj_pVal = `Adjusted P-value`)

table(reduced_LLM_genes_DF[,c("LLM_success_TF", "Source")])